In [62]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

import numpy as np
import pandas as pd
import pylab as pl
import math

from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score

%matplotlib inline 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 
import matplotlib.mlab as mlab

In [46]:
df = pd.read_csv('car_sc.csv',header=0)
df.head()

,buying,maint,doors,persons,lug_boot,safety,classe
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [47]:
##   Categorical Conversion

door_tmp = df['doors']
persons_tmp = df['persons']


X = df[['buying', 'maint', 'lug_boot', 'safety','classe']].values

ordinal_le = LabelEncoder()
X[:,0] = ordinal_le.fit_transform(X[:,0])
X[:,1] = ordinal_le.fit_transform(X[:,1])
X[:,2] = ordinal_le.fit_transform(X[:,2])
X[:,3] = ordinal_le.fit_transform(X[:,3])
X[:,4] = ordinal_le.fit_transform(X[:,4])

door_tmp = pd.get_dummies(door_tmp)
persons_tmp = pd.get_dummies(persons_tmp)


In [48]:
df = pd.DataFrame(X)
df.columns = ['buying', 'maint', 'lug_boot', 'safety', 'classe']
df[['doors_2','doors_3','doors_4','doors_5more']] = door_tmp
df[['persons_2','persons_4','persons_more']] = persons_tmp

# # Moving classe column to the end
classe_tmp = df['classe']
del df['classe']
df['classe'] = classe_tmp

df.head()

,buying,maint,lug_boot,safety,doors_2,doors_3,doors_4,doors_5more,persons_2,persons_4,persons_more,classe
0,3,3,2,1,1,0,0,0,1,0,0,2
1,3,3,2,2,1,0,0,0,1,0,0,2
2,3,3,2,0,1,0,0,0,1,0,0,2
3,3,3,1,1,1,0,0,0,1,0,0,2
4,3,3,1,2,1,0,0,0,1,0,0,2


In [49]:
df.head().values

array([[3, 3, 2, 1, 1, 0, 0, 0, 1, 0, 0, 2],
       [3, 3, 2, 2, 1, 0, 0, 0, 1, 0, 0, 2],
       [3, 3, 2, 0, 1, 0, 0, 0, 1, 0, 0, 2],
       [3, 3, 1, 1, 1, 0, 0, 0, 1, 0, 0, 2],
       [3, 3, 1, 2, 1, 0, 0, 0, 1, 0, 0, 2]], dtype=object)

In [50]:
print(df.describe())
print(df.classe.unique())

           doors_2      doors_3      doors_4  doors_5more    persons_2  \
count  1728.000000  1728.000000  1728.000000  1728.000000  1728.000000   
mean      0.250000     0.250000     0.250000     0.250000     0.333333   
std       0.433138     0.433138     0.433138     0.433138     0.471541   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     0.000000     0.000000     0.000000   
75%       0.250000     0.250000     0.250000     0.250000     1.000000   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

         persons_4  persons_more  
count  1728.000000   1728.000000  
mean      0.333333      0.333333  
std       0.471541      0.471541  
min       0.000000      0.000000  
25%       0.000000      0.000000  
50%       0.000000      0.000000  
75%       1.000000      1.000000  
max       1.000000      1.000000  
[2 0 3 1]


In [51]:
# a=df['classe']
# plt.hist(a)
# plt.title('Classe (0, 1, 2, 3)')
# plt.show()

In [52]:
df.isnull().sum()

buying          0
maint           0
lug_boot        0
safety          0
doors_2         0
doors_3         0
doors_4         0
doors_5more     0
persons_2       0
persons_4       0
persons_more    0
classe          0
dtype: int64

In [53]:
label = df['classe']
# print(label)
features = df.drop('classe', 1)
features.head()

,buying,maint,lug_boot,safety,doors_2,doors_3,doors_4,doors_5more,persons_2,persons_4,persons_more
0,3,3,2,1,1,0,0,0,1,0,0
1,3,3,2,2,1,0,0,0,1,0,0
2,3,3,2,0,1,0,0,0,1,0,0
3,3,3,1,1,1,0,0,0,1,0,0
4,3,3,1,2,1,0,0,0,1,0,0


In [54]:
#SVM

In [61]:
X = np.array(features)
y = np.array(label).astype('int')

# Hold Out
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# K-fold
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    svm=SVC(C=100,kernel='rbf', max_iter=5000, tol=1e-05, verbose=0)
    svm=svm.fit(X_train,y_train)
    scores.append(svm.score(X_test,y_test))
    
scores



/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your dat

array([ 0.84393064,  0.97687861,  0.93641618,  0.99421965,  0.93641618,
        0.97687861,  0.84393064,  0.96531792,  0.87790698,  0.96511628])

In [32]:
accuracy = {}

for i in range(1,170):
    svm=SVC(C=i,kernel='rbf', max_iter=5000, tol=1e-05, verbose=0)
    svm=svm.fit(X_train,y_train)
    performance = svm.score(X_test,y_test)
    if performance in list(accuracy.keys()):
        accuracy[performance].append(i)
    else:
        accuracy[performance]=[]
        accuracy[performance].append(i)


/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [51]:
min_performance = min(accuracy.keys())
max_performance = max(accuracy.keys())

print("Performance list: %s" % list(accuracy.keys()))
print("Min Performance: %s for C = %s" % (min_performance,accuracy[min_performance]))
print("Max Performance: %s for C = %s" % (max_performance,accuracy[max_performance]))



Performance list: [0.83101851851851849, 0.88888888888888884, 0.92592592592592593, 0.93981481481481477, 0.94907407407407407, 0.95833333333333337, 0.96064814814814814, 0.97222222222222221, 0.98148148148148151, 0.97916666666666663, 0.98379629629629628, 0.97685185185185186, 0.98611111111111116, 0.98842592592592593, 0.99305555555555558, 0.9907407407407407]
Min Performance: 0.831018518519 for C = [1]
Max Performance: 0.993055555556 for C = [117, 119, 121, 123, 124, 125, 126, 128, 130, 131, 132, 133, 134, 135, 137, 138, 141, 142, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 158, 159, 160, 161, 163, 165, 166, 167, 168, 169]


In [52]:
print('dual coef \n', svm.dual_coef_)
print ('support vectors \n', svm.support_vectors_)
print('index of support vectors \n ', svm.support_)
print ('bias', svm.intercept_)
print('the classifier \n', svm)


dual coef 
 [[ 28.33187186   0.          16.67216496 ...,  -0.         -24.39958112
   -0.        ]
 [  0.          51.44583111   0.         ...,  -0.          -0.48981804
   -7.68955324]
 [  0.           0.           0.56889671 ...,  -2.73472001  -3.99555903
   -0.        ]]
support vectors 
 [[ 2.  2.  2. ...,  0.  0.  1.]
 [ 1.  3.  2. ...,  0.  0.  1.]
 [ 2.  2.  2. ...,  0.  1.  0.]
 ..., 
 [ 1.  2.  1. ...,  0.  0.  1.]
 [ 2.  2.  0. ...,  0.  0.  1.]
 [ 1.  1.  1. ...,  0.  0.  1.]]
index of support vectors 
  [   8   33   46   51   52   62   85  128  130  133  137  139  146  147  153
  158  160  165  192  195  202  209  237  268  269  280  295  307  314  328
  338  353  365  376  379  392  393  405  413  417  428  436  437  466  483
  489  490  506  516  523  534  536  540  564  572  587  593  619  659  694
  709  722  734  739  746  769  773  783  786  789  794  804  809  814  837
  843  862  867  870  893  905  946  951  959  979  983  985  993  999 1008
 1024 1027 1039 1058 

In [49]:
print(svm.predict([[3, 3, 2, 1, 1, 0, 0, 0, 1, 0, 0],
                   [3, 3, 2, 2, 1, 0, 0, 0, 1, 0, 0],
                   [3, 3, 2, 0, 1, 0, 0, 0, 1, 0, 0],
                   [3, 3, 1, 1, 1, 0, 0, 0, 1, 0, 0],
                   [3, 3, 1, 2, 1, 0, 0, 0, 1, 0, 0]]))

[2 2 2 2 2]
